In [1]:
#Goals:
#test usability of custom Python functions in tf.Operator
#are Python library methods allowed?
#what about runtimes from other languages?
#what about arbitrary runtimes?

In [2]:
import tensorflow as tf
from multiprocessing import Process
import numpy as np
import time
import socket, sys
from sympy import *

In [3]:
tf.executing_eagerly()

True

In [4]:
#making a tf function:
@tf.function
def xor(x,y):
    return x ^ y
tf.print(xor(0,1))

1


In [5]:
#with a library:
#example from sympy documentation
#link
#https://docs.sympy.org/latest/tutorial/basic_operations.html

@tf.function
def floating_point_sqrt(x):
    expr = sqrt(8)
    return tf.convert_to_tensor(value=expr.evalf(), dtype=tf.float64)
tf.print(floating_point_sqrt(7))

2.8284271247461903


In [6]:
#arbitrary runtimes
#using docker container
#map post-activation (binary step function, relu, or leaky relu) data values to individual functions
#1 function per container
#alpine container
#put all containers into a tensorflow function
#tensorflow function implements port mapping
#docker container reads from ports, processes, and returns data

#assume the containers are already running

In [8]:
#use tensorflow layer as client, container as server

#make function able to send any piece of a tensor to a port
#run data transmissions (tensor elements -> container) in parallel
    #get tensor, split up into pieces
    #process for each thread: send to container, given socket, data point, host and port

def send_udp(socket_object, msg, recipient_tuple):
    socket_object.sendto(msg, recipient_tuple)
    return 0

def send_to_container(post_activation_tensor, ports_list, host):
    #test tensor for checking dtype
    string_tensor = tf.constant("test string")
    # create dgram udp socket
    try:
        s = socket.socket(socket.AF_INET, socket.SOCK_DGRAM)
    except socket.error:
	    print('Failed to create socket')
	    sys.exit()

    #add exception to curb mismatched datatypes in tensor
    if post_activation_tensor.dtype == string_tensor.dtype:
        msg = tf.get_static_value(post_activation_tensor)
    else:
        msg = str.encode(str(tf.get_static_value(post_activation_tensor)))

    for port in ports_list:
        s.sendto(msg, (host, port))
    return 0

send_to_container(post_activation_tensor=tf.constant("bruh"), ports_list=[50007], host='localhost')
time.sleep(2)
send_to_container(post_activation_tensor=tf.constant([1,2,1,4]), ports_list=[50007], host='localhost')

0

In [36]:
generic_tensor = tf.constant("bruh")
generic_number_tensor = tf.constant([1,2,3,4])
print(type(tf.get_static_value(generic_tensor)))

string_tensor = tf.constant("test string")
bytes_tensor = tf.constant(b'bruh')

print((generic_tensor.dtype) == (string_tensor.dtype))
print(generic_tensor.dtype)
print(string_tensor.dtype)
print(bytes_tensor.dtype)
print(type(generic_number_tensor))

<class 'bytes'>
True
<dtype: 'string'>
<dtype: 'string'>
<dtype: 'string'>
<class 'tensorflow.python.framework.ops.EagerTensor'>


In [13]:
print(type('string'))

<class 'str'>


In [1]:
tf.Tensor([1,1,1])

NameError: name 'tf' is not defined